In [1]:
!which python

/Users/karthikmaddukuri/anaconda3/envs/iris/bin/python


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import date
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import os
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
%matplotlib inline

ModuleNotFoundError: No module named 'date'

# Load Data

In [ ]:
data = datasets.load_iris()

In [ ]:
data.keys()

In [ ]:
print(data['data'][:5], end='\n\n')
print(data['feature_names'])

 _The goal is to predict the species of the flower or classify the species into setosa, versicolor, or virginica using the 4 attributes._
 
**This is a  Multiclass classification problem**

## Load data into a DataFrame

In [ ]:
df = pd.DataFrame(data['data'], columns=data['feature_names'])

In [ ]:
df['target'] = data['target']

In [ ]:
df.head()

## Exploratory Data Analysis

In [ ]:
df.shape

In [ ]:
df.describe()

#### Plotting the distribution of features

In [ ]:
columns = df.columns[:4]
for col in columns:
    df[col].hist(bins=10)
    plt.title(col)
    plt.show()

#### Plotting a Univariate Relational Plot between the features and the target

In [ ]:
df['target_name'] = df['target'].map({0: 'setosa', 1: 'versicolor', 2: "virginica"})
df.head()

In [ ]:
# Saving the data to csv
df.to_csv('../data/data.csv')

In [ ]:
col_attr = df.columns[:4]
for col in col_attr:
    sns.relplot(x=f'{col}', y='target', hue='target_name', data=df)
    plt.title(f'Relational Plot for {col}', y=2)
    plt.show()

#### Pairplots

In [ ]:
_ = sns.pairplot(df, hue='target_name')

In the _petal width v/s petal length_ plot and _target v/s petal length_ plot, _setosa_ has its own cluster. It is evident from the above plots that **_setosa_** is easily differentiable.

### Splitting the data inot Training and Testing

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=1)

In [ ]:
print(f'Training Data size is {df_train.shape}', end='\n\n')
print(f'Testing Data size is {df_test.shape}')

In [ ]:
df_train.head()

## Modeling (using Numpy arrays)

In [ ]:
X_train = df_train.drop(columns=['target', 'target_name']).values
y_train = df_train['target'].values

#### Simple Manual Model

In [ ]:
def single_feature_pred(petal_length):
    '''
    Predicts the species using the petal length
    '''
    if petal_length < 2.5:
        return 0
    elif petal_length < 4.8:
        return 1
    return 2

In [ ]:
single_feature_pred(5)

In [ ]:
# petal length values
X_train[:, 2]

In [ ]:
manual_y_pred = np.array([single_feature_pred(val) for val in X_train[:,2]])

In [ ]:
manual_model_accuracy = np.mean(manual_y_pred == y_train)
print(f'The manual model accuracy is {manual_model_accuracy :.2f}')

#### Logistic Regression

In [ ]:
model = LogisticRegression()

_Splitting into training and validation data. Although the size is very small, this is for better evaluation._

In [ ]:
X_t, X_v, y_t, y_v = train_test_split(X_train, y_train, test_size=0.25)

In [ ]:
model.fit(X_t, y_t)

In [ ]:
y_pred = model.predict(X_v)

In [ ]:
y_pred

In [ ]:
np.mean(y_pred == y_v)